In [10]:
import requests
import io
import zipfile
import frontmatter

In [11]:
repo_owner = "evidentlyai"
repo_name = "docs"
branch_name = "main"

zip_url = f"https://github.com/{repo_owner}/{repo_name}/archive/refs/heads/{branch_name}.zip"
zip_response = requests.get(zip_url)

In [12]:
documents = []
with zipfile.ZipFile(io.BytesIO(zip_response.content)) as zip_archive:
    for file_path in zip_archive.namelist():
        if not file_path.endswith(('.md', '.mdx')):
            continue
        with zip_archive.open(file_path) as file:
            content = file.read().decode('utf-8')
            post = frontmatter.loads(content)
            doc = {
                'content': post.content,
                'title':post.metadata.get('title'),
                'description': post.metadata.get('description'),
                'filename': file_path.split('/', 1)[-1]
            }
            documents.append(doc)

AttributeError: module 'frontmatter' has no attribute 'loads'